# Test functions de building_blocks (no esta todo pero casi)

In [3]:
from next_sparseconvnet.utils.data_loaders import DataGen, collatefn, LabelType

In [6]:
dataset_path = '/home/mperez/NEXT_SPARSECONVNET/next_sparseconvnet/test_files/MC_dataset.h5'

In [5]:
datagen = DataGen(dataset_path, LabelType.Segmentation)

In [6]:
with datagen:
    batch = [datagen[i] for i in range(3)]
coord, ener, lab, ev = collatefn(batch)

In [7]:
coord, ener, lab, ev = collatefn(batch)

In [8]:
dim = len(coord[0]) - 1

In [9]:
coord.shape

torch.Size([333, 4])

In [13]:
import pytest
import torch
import sparseconvnet as scn

from next_sparseconvnet.utils.data_loaders import DataGen, collatefn, LabelType
from next_sparseconvnet.networks.building_blocks import * #luego aqui como el .py lo voy a guardar en la carpeta de networks creo que basta con poner . building_blocks

def test_ResidualBlock_downsample(MCdataset):
    datagen = DataGen(MCdataset, LabelType.Classification)
    with datagen:
        data = [datagen[i] for i in range(3)]
    coord, ener, lab, ev = collatefn(data)
    spatial_size = (50, 50, 50)
    dim = 3
    inplanes = 1
    kernel = 2
    stride = 2
    
    x = scn.InputLayer(dim, spatial_size)((coord, ener))
    out = ResidualBlock_downsample(inplanes, kernel, stride)(x)
    
    assert out.features.shape[1] == 2 * inplanes
    for i, size in enumerate(spatial_size):
        assert out.spatial_size[i] == (size - kernel)/stride + 1

        
def test_ResidualBlock_basic(MCdataset):
    datagen = DataGen(MCdataset, LabelType.Classification)
    with datagen:
        data = [datagen[i] for i in range(3)]
    coord, ener, lab, ev = collatefn(data)
    spatial_size = (50, 50, 50)
    dim = 3
    inplanes = 1
    kernel = 2
    
    x = scn.InputLayer(dim, spatial_size)((coord, ener))
    out = ResidualBlock_basic(inplanes, kernel)(x)
    
    assert out.features.shape[1] == inplanes
    for i, size in enumerate(spatial_size):
        assert out.spatial_size[i] == size
        
        
def test_ResidualBlock_upsample(MCdataset):
    datagen = DataGen(MCdataset, LabelType.Classification)
    with datagen:
        data = [datagen[i] for i in range(3)]
    coord, ener, lab, ev = collatefn(data)
    spatial_size = (50, 50, 50)
    dim = 3
    inplanes = 1
    outplanes = 6
    kernel = 2
    stride = 2
    
    x = scn.InputLayer(dim, spatial_size)((coord, ener))
    x = scn.SubmanifoldConvolution(dim, inplanes, outplanes, kernel, False)(x)
    
    inplanes = x.features.shape[1]
    out = ResidualBlock_upsample(inplanes, kernel, stride)(x)
    
    assert out.features.shape[1] == inplanes / 2
    for i, size in enumerate(spatial_size):
        assert out.spatial_size[i] == kernel + stride * (size - 1)
        
        
def test_ConvBNBlock(MCdataset):
    datagen = DataGen(MCdataset, LabelType.Classification)
    with datagen:
        data = [datagen[i] for i in range(3)]
    coord, ener, lab, ev = collatefn(data)
    spatial_size = (50, 50, 50)
    dim = 3
    inplanes = 1
    outplanes = 3
    kernel = 2
    stride = 2
    
    x = scn.InputLayer(dim, spatial_size)((coord, ener))
    out = ConvBNBlock(inplanes, outplanes, kernel)(x)
    out_with_stride = ConvBNBlock(inplanes, outplanes, kernel, stride = stride)(x)
    
    for i, size in enumerate(spatial_size):
        assert out.spatial_size[i] == size
        assert out_with_stride.spatial_size[i] == (size - kernel) / stride + 1
    

In [14]:
test_ResidualBlock_downsample(dataset_path)
test_ResidualBlock_basic(dataset_path)
test_ResidualBlock_upsample(dataset_path)
test_ConvBNBlock(dataset_path)

# Test function de UNet

In [15]:
import pytest
import torch
import sparseconvnet as scn

from next_sparseconvnet.utils.data_loaders import DataGen, collatefn, LabelType
from next_sparseconvnet.networks.architectures import UNet


def test_UNet(MCdataset):
    datagen = DataGen(MCdataset, LabelType.Classification)
    with datagen:
        data = [datagen[i] for i in range(3)]
    coord, ener, lab, ev = collatefn(data)
    spatial_size = (51, 51, 51)
    init_conv_nplanes = 4
    init_conv_kernel = 3
    kernel_sizes = [7, 5, 3]
    stride = 2
    basic_num = 3
    nclasses = 3
    
    net = UNet(spatial_size, init_conv_nplanes, init_conv_kernel, kernel_sizes, stride, basic_num, nclasses = nclasses) 
    
    last_basic = []
    net.basic_up[0][2].add.register_forward_hook(lambda model, input, output: last_basic.append([output.spatial_size, output.features.shape]))
        
    assert len(net.downsample) == len(kernel_sizes) - 1
    assert len(net.upsample)   == len(kernel_sizes) - 1
    assert len(net.basic_down) == len(kernel_sizes) - 1
    assert len(net.basic_up)   == len(kernel_sizes) - 1
    assert len(net.basic_down[0]) == basic_num

    out = net.forward((coord, ener))
    
    for i, size in enumerate(last_basic[0][0]):
        assert size == spatial_size[i]
    assert last_basic[0][1][1] == init_conv_nplanes
    assert out.size()[0] == coord.size()[0]
    assert out.size()[1] == nclasses

In [16]:
test_UNet(dataset_path)

# Test functions de train_utils

In [1]:
import torch
import numpy as np
from next_sparseconvnet.utils.data_loaders import DataGen, collatefn, LabelType #luego quitar todo pq este archivo esta en carpeta utils tb
from next_sparseconvnet.networks.architectures import UNet
from next_sparseconvnet.utils.train_utils import IoU, train_one_epoch_segmentation, valid_one_epoch_segmentation #aqui igual

In [2]:
dataset_path = '/home/mperez/NEXT_SPARSECONVNET/next_sparseconvnet/test_files/MC_dataset.h5'

In [3]:
def test_IoU():
    a = [0, 1, 1, 2, 0, 0, 2]
    b = [1, 1, 2, 0, 0, 1, 2]
    iou_by_hand = [1/4, 1/4, 1/3]
    iou = IoU(a, b)
    np.testing.assert_allclose(iou, iou_by_hand)
    
    n = 5
    iou = IoU(a, b, nclass = n)
    assert len(iou) == n

In [4]:
test_IoU()

In [5]:
def test_train_one_epoch_segmentation(MCdataset):
    datagen = DataGen(MCdataset, LabelType.Segmentation, nevents = 3)
    loader = torch.utils.data.DataLoader(datagen, batch_size = 1, shuffle = True, num_workers=1, collate_fn=collatefn, drop_last=True, pin_memory=False)
    
    spatial_size = (51, 51, 51)
    init_conv_nplanes = 4
    init_conv_kernel = 3
    kernel_sizes = [7, 5, 3]
    stride_sizes = [2, 2]
    basic_num = 3
    nclasses = 3

    net = UNet(spatial_size, init_conv_nplanes, init_conv_kernel, kernel_sizes, stride_sizes, basic_num, nclasses = nclasses)
    net = net.cuda()
    
    criterion = torch.nn.CrossEntropyLoss() 
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-2, betas=(0.9, 0.999), eps=1e-6, weight_decay=0)
    
    train_one_epoch_segmentation(0, net, criterion, optimizer, loader)

In [6]:
test_train_one_epoch_segmentation(dataset_path)

Train Epoch: 0	 Loss: 0.935048


In [7]:
def test_valid_one_epoch_segmentation(MCdataset):
    datagen = DataGen(MCdataset, LabelType.Segmentation, nevents = 3)
    loader = torch.utils.data.DataLoader(datagen, batch_size = 1, shuffle = True, num_workers=1, collate_fn=collatefn, drop_last=True, pin_memory=False)
    
    spatial_size = (51, 51, 51)
    init_conv_nplanes = 4
    init_conv_kernel = 3
    kernel_sizes = [7, 5, 3]
    stride_sizes = [2, 2]
    basic_num = 3
    nclasses = 3

    net = UNet(spatial_size, init_conv_nplanes, init_conv_kernel, kernel_sizes, stride_sizes, basic_num, nclasses = nclasses)
    net = net.cuda()
    
    criterion = torch.nn.CrossEntropyLoss() 
    
    valid_one_epoch_segmentation(net, criterion, loader) 

In [8]:
test_valid_one_epoch_segmentation(dataset_path)

	 Validation Loss: 1.052158
